In [1]:
from langchain_groq import ChatGroq

In [2]:
chat = ChatGroq(temperature=0, groq_api_key="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", model_name="llama-3.2-90b-vision-preview")


In [3]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://jobs.nike.com/job/R-33460")
page_data = loader.load().pop().page_content
print(page_data)

USER_AGENT environment variable not set, consider setting it to identify your requests.























Nike Careers








































Skip to main content
Open Virtual Assistant










Home


Career Areas


Total Rewards


Life@Nike


Purpose










Language





Select a Language

  Deutsch  
  English  
  Español (España)  
  Español (América Latina)  
  Français  
  Italiano  
  Nederlands  
  Polski  
  Tiếng Việt  
  Türkçe  
  简体中文  
  繁體中文  
  עִברִית  
  한국어  
  日本語  








Careers


















Close Menu







Careers






Chat






                                Home
                            



                                Career Areas
                            



                                Total Rewards
                            



                                Life@Nike
                            



                                Purpose
                            










Jordan Careers







Converse Careers










Language











Menu



Return to Previous Menu



Select a Langua

In [5]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | chat 
res = chain_extract.invoke(input={'page_data':page_data})
type(res.content)

str

In [6]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

[{'role': '(Workplace) Lead, GPS',
  'experience': '',
  'skills': '',
  'description': 'Job Location: London, London, United Kingdom, Business Operations'},
 {'role': '6 Athlète Nike (Vendeur/euse en magasin) – H / F CDI 28h',
  'experience': '',
  'skills': '',
  'description': 'Job Location: Claye, Seine-et-Marne, France, Retail Stores'},
 {'role': 'APLA, Supply Chain Sustainability, Senior',
  'experience': '',
  'skills': '',
  'description': 'Job Location: Ciudad de México, Ciudad de Mexico, Mexico, Corporate Services'},
 {'role': 'ATHLETE III',
  'experience': '',
  'skills': '',
  'description': 'Job Location: Shenyang, Liaoning, China, Retail Stores'},
 {'role': 'Abteilungsleiter/in (BOH Coach) - 37.5H - NFS Wertheim (m/w/d)',
  'experience': '',
  'skills': '',
  'description': 'Job Location: Wertheim, Baden-Wurttemberg, Germany, Retail Stores'},
 {'role': 'Abteilungsleiter/in (Coach) - 100% - NFS Landquart',
  'experience': '',
  'skills': '',
  'description': 'Job Location:

In [7]:
type(json_res)

list

In [9]:
%pip install pandas


  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
   ---------------------------------------- 0.0/11.6 MB ? eta -:--:--
   --------------- ------------------------ 4.5/11.6 MB 26.9 MB/s eta 0:00:01
   ---------------------------- ----------- 8.1/11.6 MB 22.9 MB/s eta 0:00:01
   ------------------------------- -------- 9.2/11.6 MB 16.3 MB/s eta 0:00:01
   ---------------------------------------- 11.6/11.6 MB 14.2 MB/s eta 0:00:00
Using cached pytz-2025.2-py2.py3-none-any.whl (509 kB)
Using cached tzdata-2025.2-py2.py3-none-any.whl (347 kB)
Note: you may need to restart the kernel to use updated packages.


In [10]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [11]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [15]:
job = {"skills": "Python, machine learning, SQL"}

links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}]]

In [16]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Mohan, a business development executive at AtliQ. AtliQ is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
        Remember you are Mohan, BDE at AtliQ. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | chat
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Unlock the Power of Automation with AtliQ's Expertise

Dear [Client's Name],

I came across your job description requiring expertise in Python, machine learning, and SQL, and I'm excited to introduce you to AtliQ, a leading AI & Software Consulting company that can help you achieve your goals.

At AtliQ, we specialize in developing tailored solutions that streamline business processes, drive scalability, optimize costs, and enhance overall efficiency. Our team of experts has a proven track record of delivering high-quality solutions that meet the unique needs of our clients.

Our expertise in Python and machine learning has enabled us to develop innovative solutions that drive business growth. You can explore some of our notable projects in this space at https://example.com/ml-python-portfolio. Additionally, our proficiency in Python has allowed us to create customized solutions that cater to diverse business needs, which can be seen at https://example.com/python-portfolio.

B